### Import Modules

In [33]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time


sys.path.append("../")
import fitter.data_loader as dl
import fitter.fit_manager as fm
import fitter.fitter as fit
import fitter.special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [43]:
from importlib import reload
reload(dl)
reload(sf)
reload(fit)
reload(fm)

<module 'fitter.fit_manager' from '../fitter/fit_manager.py'>

### Specify fit parameters

In [44]:
p_dict = {
    'collection_name' : 'test', 
    
    'model_info' : {
        'fit_type' : 'xpt',
        'order' : 'nnlo', # 'nlo', 'nnlo', 'nnnlo'
        'F2' : 'FpiFpi', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)

        'include_FV' : True, # max 10
        'exclude' : [], # put LECs here
        'use_bijnens_central_value' : False,
        
        # semi-nnlo corrections
        'include_alpha_s' : False,
        'include_log' : False, 
        'include_log2' : False,
        'include_sunset' : False,
        
        # nnnlo corrections
        'include_latt_n3lo' : True,
        'include_latt_n4lo' : True,
    }, 
     #'ma', 'ma-ratio', 'xpt', 'xpt-ratio', or 'poly'; experimental: 'simultaneous', 'ma-old'

    'fast_susnet' : False,
    'use_prior' : False,
    'bias_correct' : True,
    #'include_su2_isospin_corrrection' : False,
    'abbrs' : [u'a06m310L', u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : True,
    
    'save_results' : False,
    'save_pickles' : True,
    'output_name' : 'fit_results',
}

In [45]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()
phys_point_data = data_loader.get_phys_point_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(**p_dict['model_info'], collection_name=p_dict['collection_name'])
    
# Make fit
fit_manager = fm.fit_manager(fit_data, phys_point_data, prior=prior, **p_dict)

if p_dict['make_plots']:
    data_loader.save_plots(
        fit_manager.make_plots(show_error_ellipses=p_dict['show_error_ellipses']),
        output_filename=p_dict['collection_name']+'/fits/'+fit_manager.model
    )
else:
    print(fit_manager)
    
if p_dict['save_results']:
    data_loader.save_fit_info(
        fit_manager.fit_info,
        collection_name=p_dict['collection_name'], 
        save_pickles=p_dict['save_pickles']
    )

    
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

Using default prior.
Making fit...
False
{'mjs': array([0.1619(11), 0.23041(55), 0.24277(50), 0.24979(51), 0.25812(59),
       0.3080(15), 0.3185(12), 0.3185(12), 0.3185(12), 0.3344(10),
       0.34452(72), 0.35614(82), 0.4057(56), 0.4117(17), 0.4349(16),
       0.4462(14), 0.4591(24)], dtype=object),'mju': array([0.09564(82), 0.10927(86), 0.14838(64), 0.16544(56), 0.18782(47),
       0.1492(20), 0.1772(14), 0.1772(14), 0.1772(14), 0.21883(89),
       0.23038(81), 0.26784(64), 0.232(13), 0.2566(27), 0.3059(17),
       0.3307(23), 0.3598(17)], dtype=object),'mk': array([0.162048(73), 0.228707(91), 0.24107(14), 0.24696(12), 0.25524(13),
       0.30215(11), 0.31002(17), 0.31023(19), 0.31044(22), 0.32414(21),
       0.33306(16), 0.34341(14), 0.38756(14), 0.38691(21), 0.40458(25),
       0.41370(28), 0.42723(27)], dtype=object),'mpi': array([0.094560(58), 0.097903(63), 0.14072(12), 0.15787(20), 0.18116(15),
       0.08128(16), 0.13430(17), 0.13403(15), 0.13562(32), 0.18871(17),
       0.213

In [41]:
print(fit_manager.fmt_error_budget())

Partial Errors:
                  FK/Fpi
------------------------
          A_a:   0.00004
         A_aa:   0.00005
          A_k:   0.00010
          A_p:   0.00007
          L_4:   0.00007
          L_5:   0.00012
     phys:mpi:   0.00014
      phys:mk:   0.00009
phys:lam2_chi:   0.00056
         stat:   0.00355
------------------------
        total:   0.00360



In [34]:
fit_data = {
    'mpi' : 135.,
    'mk' : 495.5,
    'Fpi' : 92.2,
    'FK' : 110.5,
    'L' : np.inf, 
    'a/w0' : 0,
}
fit_data['lam2_chi'] = (4 *np.pi *fit_data['Fpi'])**2

fit_parameters = {
    'L_1' : 0.000372,
    'L_2' : 0.000493,
    'L_3' : -0.003070,
    'L_4' : 0.000089,
    'L_5' : 0.000377,
    'L_6' : 0.000011,
    'L_7' : -0.000340,
    'L_8' : 0.000294,
    'A_a' : 0.1,
    'A_k' : -3.000000,
    'A_p' : 4.000000
}


fit_manager.fk_fpi_fit_fcn(fit_data=fit_data, fit_parameters=fit_parameters, debug=True)

NameError: name 'bootstrapper' is not defined